# ARCHS4 CRISPERCAS9 analysis


In [1]:
library(here)
library(dplyr)
library(here)
library(dplyr)
library(purrr)
library(fgsea)

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
archs4_traits  <- readRDS(here('data/archs4/traits/archs4-phenoplier.rds'))
archs4_plier2_gtex <- readRDS(here('output/archs4/archs4_PLIER2_GTEx.rds'))
archs4_plier2_kegg <- readRDS(here('output/archs4/archs4_PLIER2_KEGG.rds'))
archs4_plier2_bp <- readRDS(here('output/archs4/archs4_PLIER2_BP.rds'))
archs4_plier2_cellmarker <- readRDS(here('output/archs4/archs4_PLIER2_cellMarker.rds'))

In [3]:
archs4_z <- archs4_plier2_gtex$Z
head(archs4_z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV2357,LV2358,LV2359,LV2360,LV2361,LV2362,LV2363,LV2364,LV2365,LV2366
A1BG,0.0000000,0.6680389,0.0000000,0.0000000,0,0.3423958,0.8352773,0.5315783,0.5646032,0,⋯,0.000000,0,0.0000000,0.0000000,0.4279281,0,0.3939395,0,0.1498161,0.0000000
A1BG-AS1,0.2256781,0.1520380,0.0000000,0.0000000,0,0.1582506,0.0000000,0.0000000,0.0000000,0,⋯,0.000000,0,0.2093964,0.0000000,0.9912363,0,0.3365457,0,0.0000000,0.0000000
A2M,0.0000000,0.2468708,0.0000000,0.8209911,0,0.4362828,0.0000000,0.0000000,0.0000000,0,⋯,0.311386,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0,0.0000000,0.0000000
A2M-AS1,0.0000000,0.4181505,0.2430269,0.0000000,0,0.3849213,0.0000000,0.0000000,0.0000000,0,⋯,0.000000,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0,0.0000000,0.2756402
A2ML1,0.0000000,1.1212016,0.1917428,0.3451314,0,0.0000000,0.0000000,0.0000000,0.0000000,0,⋯,0.000000,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0,0.0000000,0.0000000
A4GALT,0.0000000,0.4097771,0.0000000,1.0070209,0,0.4368749,0.0000000,0.5688822,0.7836861,0,⋯,0.000000,0,0.3276324,0.3872317,0.0000000,0,0.0000000,0,0.0000000,0.0000000


# CRISPR CAS9

In [4]:
lipid_deg <- read.csv(
  here("data/archs4/crispr_cas9/lipid_DEG.csv"),
  stringsAsFactors = FALSE
)

head(lipid_deg)

,gene_name,GFPLow_vs_UnSorted.log2FC,GFPLow_vs_UnSorted.FDR,GFPLow_vs_UnSorted.DEG,GFPHigh_vs_UnSorted.log2FC,GFPHigh_vs_UnSorted.FDR,GFPHigh_vs_UnSorted.DEG,GFPHigh_vs_GFPLow.log2FC,GFPHigh_vs_GFPLow.FDR,GFPHigh_vs_GFPLow.DEG,lipid.effect,rank
,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<lgl>,<chr>,<int>
1,ABCA2,0.55866839,0.116007580,FALSE,0.78491850,0.02712826,TRUE,0.2262501,0.632690747,FALSE,decrease *,-1
2,ABCB7,0.79852459,0.029219603,TRUE,-0.23834184,0.79077801,FALSE,-1.0368664,0.000693259,TRUE,increase **,2
3,ABCE1,0.68407544,0.047833353,TRUE,0.04432565,0.97525531,FALSE,-0.6397498,0.049228517,TRUE,increase **,2
4,ABCG2,0.43812607,0.049607475,TRUE,0.37554996,0.22953855,FALSE,-0.0625761,0.883677774,FALSE,increase *,1
5,ABHD10,-0.64140454,0.009845648,TRUE,-0.24194344,0.65855335,FALSE,0.3994611,0.156967126,FALSE,decrease *,-1
6,ABHD13,0.08890522,0.806738313,FALSE,-0.45420696,0.04021766,TRUE,-0.5431122,0.000576641,TRUE,increase **,2


In [5]:
orig_deg_gene_sets <- list()
for (r in unique(lipid_deg$rank)) {
  if (r == 0) next
  genes <- lipid_deg$gene_name[lipid_deg$rank == r]
  orig_deg_gene_sets[[paste0("gene_set_", r)]] <- genes
}

head(orig_deg_gene_sets)

$`gene_set_-1`
 [1] "ABCA2"       "ABHD10"      "ADAM32"      "ADAMTS12"    "ALKBH3"     
 [6] "APITD1-CORT" "ASB9"        "ATP5B"       "ATP6AP1"     "C12orf29"   
[11] "C16orf78"    "C21orf140"   "C5orf63"     "CBL"         "CBX1"       
[16] "CDKN1A"      "CETN3"       "COL4A5"      "COPS7A"      "DHRS7"      
[21] "GFRA4"       "GLRA3"       "GNA15"       "GPRASP1"     "GTF2A1L"    
[26] "GUCY2F"      "HELZ2"       "HEXDC"       "HRH2"        "HSBP1L1"    
[31] "IFNA4"       "IQCF2"       "KCNJ5"       "LAMC1"       "LOC645382"  
[36] "LPGAT1"      "MAML1"       "MEDAG"       "NDUFAB1"     "NDUFB8"     
[41] "NLRP3"       "NRIP1"       "OPN1LW"      "PAIP2B"      "PDCL"       
[46] "PDE4B"       "PHYHIP"      "PRR16"       "RPS6KC1"     "RUNX1"      
[51] "S100A5"      "SLAMF8"      "SLC35G3"     "SLC7A3"      "SMIM6"      
[56] "SMPD1"       "SNX13"       "SREBF1"      "STK32C"      "TADA1"      
[61] "TADA2B"      "TAS2R39"     "TCEB1"       "TDRP"        "TMBIM4"     
[66] "TMIGD1"      "TTC38"       "UGT1A5"      "VPS13A"      "VWA1"       
[71] "ZFP36L1"     "ZFP36L2"     "ZNF133"      "ZNF214"      "ZNF583"     
[76] "ZNF7"        "ZNF799"     

$gene_set_2
  [1] "ABCB7"          "ABCE1"          "ABHD13"         "ACTR1A"        
  [5] "ACVR1"          "AGPAT6"         "AKIRIN2"        "AP2S1"         
  [9] "ASCC3"          "ASH2L"          "B3GALT4"        "BDP1"          
 [13] "BMPR1A"         "C1orf109"       "C21orf59"       "C7orf26"       
 [17] "CBLL1"          "CCIN"           "CEBPA"          "CHD4"          
 [21] "CHERP"          "CHP1"           "CPNE6"          "CPSF4"         
 [25] "CRKL"           "CSTF3"          "DCAF7"          "DDTL"          
 [29] "DHX33"          "DKC1"           "DNAJC22"        "DPH1"          
 [33] "DPH3"           "E4F1"           "EIF5"           "EIF6"          
 [37] "ELL"            "ELP4"           "ELP5"           "EMG1"          
 [41] "ERCC3"          "EWSR1"          "EXOC3"          "FBL"           
 [45] "FDX1L"          "FERMT2"         "FNDC3B"         "FNTA"          
 [49] "GATAD2A"        "GNL3L"          "GPN1"           "GPN2"          
 [53] "GTF2B"          "GTF2E2"         "GTF2H1"         "HECTD1"        
 [57] "HNF4A"          "HNRNPH1"        "HNRNPL"         "HSP90B1"       
 [61] "IGF1R"          "IMP4"           "INTS1"          "INTS6"         
 [65] "IPO13"          "IPO7"           "ISY1"           "ITGB1"         
 [69] "KIAA1429"       "KMT2D"          "LSM12"          "LSM4"          
 [73] "LUC7L3"         "MAK16"          "MBTPS2"         "MDM2"          
 [77] "MED8"           "NAPA"           "NARFL"          "NHP2"          
 [81] "NMD3"           "NOL6"           "NOP10"          "NOP16"         
 [85] "NOP2"           "NOP56"          "NR5A1"          "ORAOV1"        
 [89] "OSBP"           "OTUD5"          "PCBP1"          "PDCD2"         
 [93] "PFDN2"          "PFDN6"          "PHF20"          "PLIN2"         
 [97] "POLR2C"         "POLR2F"         "POLR2K"         "POLR2L"        
[101] "POLR3E"         "POP7"           "PPP1R11"        "PWP2"          
[105] "QPCTL"          "RANGAP1"        "RAP1GDS1"       "RBBP6"         
[109] "RPAP3"          "RPL13"          "RPL17-C18orf32" "RPL18"         
[113] "RPL19"          "RPL31"          "RPL34"          "RPL35A"        
[117] "RPL37"          "RPL6"           "RPL7"           "RPL7L1"        
[121] "RPLP0"          "RPP38"          "RPS11"          "RPS13"         
[125] "RPS14"          "RPS16"          "RPS19"          "RPS2"          
[129] "RPS27A"         "RPS28"          "RPS6"           "RPSA"          
[133] "SAFB"           "SAP130"         "SEC61G"         "SLC12A9"       
[137] "SLC33A1"        "SLC35B2"        "SNRPD1"         "SNRPD3"        
[141] "SPATA5"         "SPDYE2"         "SREBF2"         "SRP19"         
[145] "SRP54"          "SRP72"          "STAG2"          "SUDS3"         
[149] "SUPT5H"         "TAF1C"          "TAF6"           "TANGO6"        
[1

In [6]:
deg_gene_sets <- list(
  gene_set_increase = orig_deg_gene_sets[["gene_set_3"]],
  gene_set_decrease = orig_deg_gene_sets[["gene_set_-3"]]
)

head(deg_gene_sets)

$gene_set_increase
[1] "ACACA"  "DGAT2"  "HILPDA" "MBTPS1" "SCAP"   "SRPR"  

$gene_set_decrease
[1] "BLCAP"  "FBXW7"  "INSIG2" "PCYT2"  "PTEN"   "SOX9"   "TCF7L2" "UBE2J2"

In [7]:
lvs <- set_names(
  lapply(seq_len(ncol(archs4_z)), function(i) {
    vec <- archs4_z[, i]
    names(vec) <- rownames(archs4_z)
    vec
  }),
  nm = paste0("LV", seq_len(ncol(archs4_z)))
)

length(lvs)

[1] 2366

In [8]:
n_reps <- 10
set.seed(0)

In [9]:
dir.create(here("output/crispr"), showWarnings = FALSE, recursive = TRUE)
results_file <- here("output/crispr/results.rds")

if (file.exists(results_file)) {
  results <- readRDS(results_file)
} else {
  results <- imap(lvs, ~{
    lv_name <- .y
    reps <- map(seq_len(n_reps), function(i) {
      fg <- fgsea(
        pathways  = deg_gene_sets,
        stats     = .x,
        scoreType = "pos",
        eps       = 0.0
      ) %>%
        arrange(pval) %>%
        mutate(
          lv      = lv_name,
          rep_idx = i
        )
      fg
    })
    bind_rows(reps)
  })
  saveRDS(results, results_file)
}

In [10]:
crispr_cas_results <- bind_rows(results) %>%
  mutate(leadingEdge = map_chr(leadingEdge, toString))

In [11]:
crispr_cas_results_sig  <- crispr_cas_results  %>% 
dplyr::filter(padj < 0.05)

In [12]:
dim(crispr_cas_results_sig)

[1] 411  10

In [13]:
head(crispr_cas_results_sig)

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
gene_set_decrease,0.01847419,0.03694839,0.3524879,0.9706902,1.219636,8,"SOX9, INSIG2",LV4,1
gene_set_decrease,0.02358078,0.04716156,0.3524879,0.9706902,1.235971,8,"SOX9, INSIG2",LV4,2
gene_set_decrease,0.02418156,0.04836311,0.3524879,0.9706902,1.224327,8,"SOX9, INSIG2",LV4,3
gene_set_decrease,0.01516995,0.03033989,0.3807304,0.9706902,1.216527,8,"SOX9, INSIG2",LV4,4
gene_set_decrease,0.02497502,0.04995005,0.2878571,0.9706902,1.234782,8,"SOX9, INSIG2",LV4,5
gene_set_decrease,0.02237923,0.04475846,0.3524879,0.9706902,1.235104,8,"SOX9, INSIG2",LV4,8


In [14]:
crispr_cas_results_sig_sub  <- crispr_cas_results_sig  %>% 
dplyr::rename(LV=lv)  %>% 
dplyr::rename(pathway_gene=pathway)  %>% 
dplyr::select(LV, pathway_gene, leadingEdge)

In [15]:
lv_sig <- unique(crispr_cas_results_sig_sub$LV)

# Pathways

In [18]:
archs4_plier2_gtex$summary %>%
    dplyr::filter(LV %in% lv_sig) %>%
    group_by(LV) %>%
    summarise(pathways = paste(pathway, collapse = "; ")) %>%
    arrange(as.numeric(gsub("LV", "", LV)))

archs4_plier2_kegg$summary %>%
    dplyr::filter(LV %in% lv_sig) %>%
    group_by(LV) %>%
    summarise(pathways = paste(pathway, collapse = "; ")) %>%
    arrange(as.numeric(gsub("LV", "", LV)))

archs4_plier2_bp$summary %>%
    dplyr::filter(LV %in% lv_sig) %>%
    group_by(LV) %>%
    summarise(pathways = paste(pathway, collapse = "; ")) %>%
    arrange(as.numeric(gsub("LV", "", LV)))

archs4_plier2_cellmarker$summary %>%
        dplyr::filter(LV %in% lv_sig) %>%
        group_by(LV) %>%
        summarise(pathways = paste(pathway, collapse = "; ")) %>%
        arrange(as.numeric(gsub("LV", "", LV)))

LV,pathways
<chr>,<chr>
LV4,GTEx_Tissues_Nerve - Tibial Male 50-59 Up; GTEx_Tissues_Heart - Atrial Appendage Male 50-59 Up; GTEx_Tissues_Artery - Tibial Male 50-59 Up; GTEx_Tissues_Artery - Coronary Male 20-29 Up; GTEx_Tissues_Nerve - Tibial Male 70-79 Up; GTEx_Tissues_Heart - Atrial Appendage Female 40-49 Up; GTEx_Tissues_Artery - Tibial Male 70-79 Up; GTEx_Tissues_Esophagus - Gastroesophageal Junction Male 70-79 Up; GTEx_Tissues_Artery - Aorta Female 70-79 Up; GTEx_Tissues_Cervix - Ectocervix Female 40-49 Up
LV37,GTEx_Tissues_Brain - Caudate (Basal Ganglia) Male 50-59 Up; GTEx_Tissues_Brain - Putamen (Basal Ganglia) Male 50-59 Up; GTEx_Tissues_Pituitary Male 40-49 Up; GTEx_Tissues_Brain - Cerebellar Hemisphere Male 20-29 Up; GTEx_Tissues_Brain - Caudate (Basal Ganglia) Male 70-79 Up; GTEx_Tissues_Brain - Putamen (Basal Ganglia) Female 40-49 Up
LV38,GTEx_Tissues_Colon - Transverse Male 50-59 Up; GTEx_Tissues_Colon - Transverse Male 60-69 Up; GTEx_Tissues_Kidney - Cortex Male 60-69 Up; GTEx_Tissues_Esophagus - Muscularis Male 40-49 Up; GTEx_Tissues_Esophagus - Gastroesophageal Junction Male 40-49 Up; GTEx_Tissues_Pancreas Female 40-49 Up; GTEx_Tissues_Brain - Cortex Female 50-59 Up; GTEx_Tissues_Cervix - Endocervix Female 40-49 Up
LV53,GTEx_Tissues_Esophagus - Mucosa Male 30-39 Up
LV100,GTEx_Tissues_Lung Male 50-59 Up; GTEx_Tissues_Lung Female 60-69 Up; GTEx_Tissues_Pancreas Female 60-69 Up; GTEx_Tissues_Lung Male 30-39 Up; GTEx_Tissues_Lung Female 50-59 Up; GTEx_Tissues_Lung Male 20-29 Up; GTEx_Tissues_Lung Female 30-39 Up; GTEx_Tissues_Heart - Atrial Appendage Female 40-49 Up; GTEx_Tissues_Minor Salivary Gland Male 40-49 Up; GTEx_Tissues_Lung Female 20-29 Up
LV165,GTEx_Tissues_Heart - Atrial Appendage Female 60-69 Up; GTEx_Tissues_Ovary Female 60-69 Up; GTEx_Tissues_Adrenal Gland Female 50-59 Up; GTEx_Tissues_Spleen Female 50-59 Up; GTEx_Tissues_Adrenal Gland Male 40-49 Up; GTEx_Tissues_Adrenal Gland Male 70-79 Up
LV181,GTEx_Tissues_Pancreas Male 60-69 Up; GTEx_Tissues_Adipose - Subcutaneous Male 40-49 Up; GTEx_Tissues_Cells - Cultured Fibroblasts Male 30-39 Up; GTEx_Tissues_Thyroid Male 30-39 Up; GTEx_Tissues_Nerve - Tibial Male 20-29 Up; GTEx_Tissues_Pancreas Male 40-49 Up; GTEx_Tissues_Skin - Sun Exposed (Lower Leg) Male 70-79 Up; GTEx_Tissues_Artery - Aorta Male 70-79 Up
LV239,GTEx_Tissues_Whole Blood Female 60-69 Up; GTEx_Tissues_Pituitary Male 50-59 Up; GTEx_Tissues_Brain - Hypothalamus Male 60-69 Up; GTEx_Tissues_Whole Blood Male 30-39 Up; GTEx_Tissues_Cells - Cultured Fibroblasts Male 30-39 Up; GTEx_Tissues_Liver Female 60-69 Up; GTEx_Tissues_Cells - Cultured Fibroblasts Female 30-39 Up; GTEx_Tissues_Whole Blood Male 20-29 Up; GTEx_Tissues_Whole Blood Female 70-79 Up
LV282,GTEx_Tissues_Whole Blood Male 50-59 Up; GTEx_Tissues_Pituitary Male 50-59 Up; GTEx_Tissues_Whole Blood Male 40-49 Up


LV,pathways
<chr>,<chr>
LV4,KEGG_Axon guidance; KEGG_Basal cell carcinoma; KEGG_Breast cancer; KEGG_Circadian entrainment; KEGG_Complement and coagulation cascades; KEGG_Cortisol synthesis and secretion; KEGG_ECM-receptor interaction; KEGG_Protein digestion and absorption; KEGG_Vascular smooth muscle contraction
LV37,KEGG_Axon guidance; KEGG_Gastric acid secretion
LV38,KEGG_GABAergic synapse; KEGG_Ribosome; KEGG_Ribosome biogenesis in eukaryotes
LV53,KEGG_Alcoholism; KEGG_Coronavirus disease; KEGG_Herpes simplex virus 1 infection; KEGG_Legionellosis; KEGG_Proteasome; KEGG_Ribosome; KEGG_Taste transduction; KEGG_Viral carcinogenesis
LV100,"KEGG_AGE-RAGE signaling pathway in diabetic complications; KEGG_Complement and coagulation cascades; KEGG_Cytokine-cytokine receptor interaction; KEGG_ECM-receptor interaction; KEGG_Lysosome; KEGG_Parathyroid hormone synthesis, secretion and action; KEGG_Proximal tubule bicarbonate reclamation; KEGG_Retinol metabolism; KEGG_Vascular smooth muscle contraction"
LV165,KEGG_Amino sugar and nucleotide sugar metabolism; KEGG_Autophagy; KEGG_Calcium signaling pathway; KEGG_Circadian rhythm; KEGG_Endocytosis; KEGG_Ferroptosis; KEGG_Lysosome; KEGG_Mineral absorption; KEGG_Rheumatoid arthritis; KEGG_mTOR signaling pathway
LV181,KEGG_Central carbon metabolism in cancer; KEGG_Fructose and mannose metabolism; KEGG_Glycolysis / Gluconeogenesis; KEGG_HIF-1 signaling pathway; KEGG_Nitrogen metabolism
LV239,KEGG_Cortisol synthesis and secretion; KEGG_Estrogen signaling pathway; KEGG_Hematopoietic cell lineage; KEGG_Inflammatory bowel disease; KEGG_Leukocyte transendothelial migration; KEGG_Nitrogen metabolism; KEGG_Pantothenate and CoA biosynthesis; KEGG_Proximal tubule bicarbonate reclamation; KEGG_Starch and sucrose metabolism
LV255,KEGG_Breast cancer; KEGG_Cocaine addiction; KEGG_Dilated cardiomyopathy; KEGG_RNA transport; KEGG_Spliceosome; KEGG_Synaptic vesicle cycle


LV,pathways
<chr>,<chr>
LV4,BP_Cellular Response to BMP Stimulus (GO:0071773); BP_Negative Regulation of Actin Filament Bundle Assembly (GO:0032232); BP_Positive Regulation of Bone Mineralization (GO:0030501); BP_Regulation of Cell Migration Involved in Sprouting Angiogenesis (GO:0090049); BP_Regulation of Vascular Associated Smooth Muscle Cell Proliferation (GO:1904705); BP_Transforming Growth Factor Beta Receptor Signaling Pathway (GO:0007179)
LV38,BP_Anterior/Posterior Pattern Specification (GO:0009952); BP_Negative Regulation of Myeloid Leukocyte Differentiation (GO:0002762); BP_Neural Crest Cell Migration (GO:0001755); BP_Positive Regulation of Nuclear Division (GO:0051785); BP_Transport Across Blood-Brain Barrier (GO:0150104)
LV53,BP_Positive Regulation of Transcription Initiation by RNA Polymerase II (GO:0060261); BP_Secondary Alcohol Biosynthetic Process (GO:1902653)
LV165,BP_Chloride Transport (GO:0006821); BP_Cholesterol Homeostasis (GO:0042632); BP_Glycosphingolipid Catabolic Process (GO:0046479); BP_Negative Regulation of Organelle Organization (GO:0010639); BP_Negative Regulation of Protein Polymerization (GO:0032272); BP_Positive Regulation of Proteasomal Protein Catabolic Process (GO:1901800); BP_Positive Regulation of Response to External Stimulus (GO:0032103); BP_Regulation of TORC1 Signaling (GO:1903432); BP_Regulation of Brown Fat Cell Differentiation (GO:0090335)
LV181,BP_Cellular Response to Hypoxia (GO:0071456); BP_Cellular Response to Sterol (GO:0036315); BP_Granulocyte Chemotaxis (GO:0071621); BP_Negative Regulation of Cellular Response to Growth Factor Stimulus (GO:0090288); BP_Positive Regulation of Cold-Induced Thermogenesis (GO:0120162); BP_Regulation of Glycogen Biosynthetic Process (GO:0005979)
LV442,BP_Gas Transport (GO:0015669); BP_Hydrogen Peroxide Catabolic Process (GO:0042744); BP_Response to Hydrogen Peroxide (GO:0042542)
LV455,BP_Extracellular Structure Organization (GO:0043062)
LV587,"BP_Aerobic Electron Transport Chain (GO:0019646); BP_Intrinsic Apoptotic Signaling Pathway in Response to DNA Damage by P53 Class Mediator (GO:0042771); BP_Negative Regulation of Response to External Stimulus (GO:0032102); BP_Oxidative Phosphorylation (GO:0006119); BP_Regulation of Fibroblast Proliferation (GO:0048145); BP_Regulation of Protein Localization to Plasma Membrane (GO:1903076); BP_Signal Transduction by P53 Class Mediator (GO:0072331); BP_Wound Healing, Spreading of Cells (GO:0044319)"
LV773,BP_Embryonic Heart Tube Morphogenesis (GO:0003143); BP_Extracellular Structure Organization (GO:0043062); BP_Positive Regulation of Tyrosine Phosphorylation of STAT Protein (GO:0042531); BP_Regulation of Collagen Biosynthetic Process (GO:0032965)


LV,pathways
<chr>,<chr>
LV4,CellMarker_DCLK1+ Progenitor Cell Large Intestine Human; CellMarker_Fibroblast Skin Mouse; CellMarker_Glial Cell Colon Human; CellMarker_Hematopoietic Stem Cell Bone Marrow Mouse; CellMarker_Mesenchymal Stem Cell Bone Human; CellMarker_Podocyte Kidney Mouse; CellMarker_Schwalie Et al.Nature.P3 Adipose Tissue Mouse; CellMarker_Vascular Endothelial Cell Kidney Mouse
LV53,CellMarker_Microglial Cell White Matter Mouse; CellMarker_Morula Cell (Blastomere) Embryo Human; CellMarker_SLC16A7+ Cell Lung Human; CellMarker_Schwalie Et al.Nature.G4 Adipose Tissue Mouse; CellMarker_T Cell Kidney Mouse
LV100,CellMarker_Cancer Cell Lung Human; CellMarker_Clara Cell Lung Mouse; CellMarker_Epithelial Cell Bronchial Vessel Human; CellMarker_Epithelial Cell Lung Human; CellMarker_Epithelial Cell Lung Mouse; CellMarker_Secretory Cell Lung Human; CellMarker_Stem Cell Lung Human; CellMarker_Type II Pneumocyte Lung Mouse
LV165,CellMarker_Macrophage Nasopharynx Human; CellMarker_Microglial Cell Brain Mouse; CellMarker_Microglial Cell Hippocampus Mouse; CellMarker_Paneth Cell Large Intestine Human
LV181,CellMarker_Brush Cell (Tuft Cell) Lung Human; CellMarker_CD4+ T Cell Peripheral Blood Human; CellMarker_Fibroblast Undefined Human; CellMarker_Granulosa Cell Fetal Gonad Human; CellMarker_Matrix Fibroblast Lung Mouse; CellMarker_Tumor Associated Macrophage (TAM) Breast Human
LV282,CellMarker_Cancer Stem Cell Pancreas Human; CellMarker_Fibrocartilage Chondrocyte Articular Cartilage Human; CellMarker_Mesenchymal Cell Liver Human; CellMarker_Myeloid Cell Lung Mouse; CellMarker_Plasmacytoid Dendritic Cell Spleen Mouse
LV339,CellMarker_B Cell Spleen Mouse; CellMarker_Exhausted T(Tex) Cell Undefined Human; CellMarker_Regulatory T(Treg) Cell Liver Human
LV442,CellMarker_M1 Macrophage Lung Human; CellMarker_Red Blood Cell (Erythrocyte) Bone Marrow Human
LV455,CellMarker_Dendritic Cell Stomach Human; CellMarker_Osteocyte Bone Human; CellMarker_Red Blood Cell (Erythrocyte) Bone Marrow Human


In [18]:
merged_df <- archs4_plier2_kegg$summary %>%
    dplyr::filter(LV %in% lv_sig) %>%
    group_by(LV) %>%
    summarise(kegg_pathways = paste(pathway, collapse = "; ")) %>%
    left_join(
        archs4_plier2_bp$summary %>%
            dplyr::filter(LV %in% lv_sig) %>%
            group_by(LV) %>%
            summarise(bp_pathways = paste(pathway, collapse = "; ")),
        by = "LV"
    ) %>%
    left_join(
        archs4_plier2_cellmarker$summary %>%
            dplyr::filter(LV %in% lv_sig) %>%
            group_by(LV) %>%
            summarise(cellmarker_pathways = paste(pathway, collapse = "; ")),
        by = "LV"
    )

merged_df %>%
    filter(
        grepl("liver", kegg_pathways, ignore.case = TRUE) |
        grepl("liver", bp_pathways, ignore.case = TRUE) |
        grepl("liver", cellmarker_pathways, ignore.case = TRUE)
    )

LV,kegg_pathways,bp_pathways,cellmarker_pathways
<chr>,<chr>,<chr>,<chr>
LV1209,KEGG_ABC transporters; KEGG_Arrhythmogenic right ventricular cardiomyopathy; KEGG_Hepatocellular carcinoma; KEGG_Measles; KEGG_PPAR signaling pathway; KEGG_Pancreatic secretion; KEGG_Renin secretion; KEGG_Staphylococcus aureus infection; KEGG_cAMP signaling pathway,BP_Fc-gamma Receptor Signaling Pathway Involved in Phagocytosis (GO:0038096); BP_Negative Regulation of Interleukin-10 Production (GO:0032693); BP_Positive Regulation of Bone Mineralization (GO:0030501); BP_Positive Regulation of Interleukin-10 Production (GO:0032733); BP_Positive Regulation of Lamellipodium Organization (GO:1902745); BP_Regulation of Tumor Necrosis Factor Production (GO:0032680); BP_Transport Across Blood-Brain Barrier (GO:0150104),CellMarker_Dendritic Cell Stomach Human; CellMarker_Dendritic Cell Undefined Human; CellMarker_Exhausted T(Tex) Cell Liver Human; CellMarker_Monocyte-derived Dendritic Cell Undefined Human; CellMarker_Monocyte Kidney Mouse; CellMarker_Myeloid Cell Bone Marrow Human; CellMarker_Myeloid Cell Lung Mouse; CellMarker_Neutrophil Kidney Human
LV1704,KEGG_Arachidonic acid metabolism; KEGG_Butanoate metabolism; KEGG_Cholinergic synapse; KEGG_Complement and coagulation cascades; KEGG_ECM-receptor interaction; KEGG_Epithelial cell signaling in Helicobacter pylori infection; KEGG_Transcriptional misregulation in cancer; KEGG_Vascular smooth muscle contraction; KEGG_Wnt signaling pathway,BP_Extracellular Structure Organization (GO:0043062); BP_Negative Regulation of Endothelial Cell Apoptotic Process (GO:2000352); BP_Neutrophil Chemotaxis (GO:0030593); BP_Positive Regulation of Endothelial Cell Apoptotic Process (GO:2000353); BP_Pulmonary Valve Development (GO:0003177); BP_Regulation of Interleukin-2 Production (GO:0032663); BP_Regulation of Vascular Associated Smooth Muscle Cell Proliferation (GO:1904705); BP_Tube Closure (GO:0060606),CellMarker_CD8+ Recently Activated Effector Memory Or Effector T Cell (CTL) Blood Human; CellMarker_Endothelial Cell Embryo Mouse; CellMarker_MKI67+ T Cell Bile Duct Human; CellMarker_Macrophage Kidney Human; CellMarker_Progenitor Cell Liver Human; CellMarker_Smooth Muscle Cell Lung Mouse; CellMarker_Stromal Cell Colon Human; CellMarker_T Cell Ovary Human
LV1954,KEGG_Cholesterol metabolism; KEGG_Complement and coagulation cascades; KEGG_Cytokine-cytokine receptor interaction; KEGG_Glycosaminoglycan biosynthesis; KEGG_Maturity onset diabetes of the young; KEGG_Morphine addiction; KEGG_Taurine and hypotaurine metabolism; KEGG_Thyroid hormone synthesis; KEGG_Vitamin digestion and absorption; KEGG_cAMP signaling pathway,NA,CellMarker_Enterocyte Progenitor Cell Small Intestine Human; CellMarker_Hepatocyte Liver Human; CellMarker_Hepatocyte Liver Mouse; CellMarker_Liver Bud Hepatic Cell Liver Human; CellMarker_Mesenchymal Stem Cell Muscle Human; CellMarker_Primitive Endoderm Cell Embryo Human; CellMarker_T Cell Bone Marrow Mouse
LV2007,"KEGG_Arrhythmogenic right ventricular cardiomyopathy; KEGG_Cell adhesion molecules; KEGG_Cholesterol metabolism; KEGG_Inositol phosphate metabolism; KEGG_Mineral absorption; KEGG_Parathyroid hormone synthesis, secretion and action; KEGG_Pathways in cancer; KEGG_Regulation of lipolysis in adipocytes",BP_Ventricular Cardiac Muscle Tissue Morphogenesis (GO:0055010),CellMarker_Epithelial Cell Epithelium Human; CellMarker_Hematopoietic Stem Cell Bone Marrow Human; CellMarker_Induced Pluripotent Stem Cell Undefined Human; CellMarker_Progenitor Cell Liver Human; CellMarker_Trophectoderm Cell Embryo Human
LV282,KEGG_Diabetic cardiomyopathy; KEGG_Ferroptosis; KEGG_Human immunodeficiency virus 1 infection; KEGG_Hypertrophic cardiomyopathy; KEGG_Oxidative phosphorylation; KEGG_Protein processing in endoplasmic reticulum; KEGG_Salivary secretion; KEGG_Spliceosome; KEGG_Synaptic vesicle cycle; KEGG_Vasopressin-regulated water reabsorption,NA,CellMarker_Cancer Stem Cell Pancreas Human; CellMarker_Fibrocartilage

# top 1% between different models and chose one